In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [8]:
gs = GridSearchCV(NMF, {"n_epochs": [10, 20], "reg_bu": [0.01, 0.1, 0.2], "reg_bi": [0.01, 0.1, 0.2]}, measures=["rmse"])
gs.fit(Dataset.load_builtin('ml-100k'))
print(gs.best_score["rmse"], gs.best_params["rmse"])

0.9767450058764539 {'n_epochs': 20, 'reg_bu': 0.1, 'reg_bi': 0.01}
